In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point

In [2]:
policeData_17 = pd.read_csv('../Data/police_data/2017clean.csv', index_col=0)

policeData_16 = pd.read_csv('../Data/police_data/2016clean.csv', index_col=0)

policeData_15 = pd.read_csv('../Data/police_data/2015clean.csv', index_col=0)

policeData_14 = pd.read_csv('../Data/police_data/2014clean.csv', index_col=0)

policeData_13 = pd.read_csv('../Data/police_data/2013clean.csv', index_col=0)

#geocodes = pd.read_csv('../Data/police_data/cop_geocodes.csv', index_col=0)

In [3]:
geocode1 = pd.read_csv('../Data/police_data/geocodes/Cop_geocodes_991.csv', index_col=0)
geocode2 = pd.read_csv('../Data/police_data/geocodes/Cop_geocodes_991_2000.csv', index_col=0)
geocode3 = pd.read_csv('../Data/police_data/geocodes/Cop_geocodes_1984_3000.csv', index_col=0)
geocode4 = pd.read_csv('../Data/police_data/geocodes/Cop_geocodes_2993_4000.csv', index_col=0)
geocode5 = pd.read_csv('../Data/police_data/geocodes/Cop_geocodes_3988_5000.csv', index_col=0)
geocode6 = pd.read_csv('../Data/police_data/geocodes/Cop_geocodes_4989_6000.csv', index_col=0)
geocode7 = pd.read_csv('../Data/police_data/geocodes/Cop_geocodes_5985_7000.csv', index_col=0)
geocode8 = pd.read_csv('../Data/police_data/geocodes/Cop_geocodes_6971_8000.csv', index_col=0)
geocode9 = pd.read_csv('../Data/police_data/geocodes/Cop_geocodes_7972_9000.csv', index_col=0)
geocode10 = pd.read_csv('../Data/police_data/geocodes/Cop_geocodes_8982.csv', index_col=0)
geocode11 = pd.read_csv('../Data/police_data/geocodes/Cop_geocodes_10504.csv', index_col=0)

In [4]:
geocodes = geocode1.append(geocode2)
geocodes = geocodes.append(geocode3)
geocodes = geocodes.append(geocode4)
geocodes = geocodes.append(geocode5)
geocodes = geocodes.append(geocode6)
geocodes = geocodes.append(geocode7)
geocodes = geocodes.append(geocode8)
geocodes = geocodes.append(geocode9)
geocodes = geocodes.append(geocode10)
geocodes = geocodes.append(geocode11)

### Create Final Dataframe

In [5]:
policeData = policeData_17.append(policeData_16)
policeData = policeData.append(policeData_15)
policeData = policeData.append(policeData_14)
policeData = policeData.append(policeData_13)

In [6]:
len(policeData)

49876

In [16]:
policeData.to_csv('../Data/police_data/police_data_13_17.csv')

### Get count for each property

In [17]:
policeData.head()

,oca,mrc,trc,apt,ltp,zne,enddate,loc1,loc2,common_nam,offtext,numbered,address1,address2
0,1700001,T,00:52:55,NaN,1.0,O,2017-01-01,3567,WOODRIDGE RD,NaN,MISSING PERSON,True,3567 Woodridge Rd,3567 & Woodridge Rd
1,1700001,T,00:52:55,NaN,1.0,O,2017-01-01,3567,WOODRIDGE RD,NaN,FOUND PERSON,True,3567 Woodridge Rd,3567 & Woodridge Rd
2,1700002,I,01:14:13,NaN,1.0,O,2017-01-01,3567,WOODRIDGE RD,NaN,MISSING PERSON,True,3567 Woodridge Rd,3567 & Woodridge Rd
3,1700002,I,01:14:13,NaN,1.0,O,2017-01-01,3567,WOODRIDGE RD,NaN,FOUND PERSON,True,3567 Woodridge Rd,3567 & Woodridge Rd
4,1700003,I,01:15:30,NaN,1.0,O,2017-01-01,3567,WOODRIDGE RD,NaN,MISSING PERSON,True,3567 Woodridge Rd,3567 & Woodridge Rd


In [18]:
policeData_count = policeData.groupby('address1').count()
policeData_count = policeData_count[['offtext']]
policeData_count.columns = ['PD_Count']
policeData_count.head()

,PD_Count
address1,
3186 Euclid Heights Blvd,2
4075 Monticello Blvd,2
1028 Greyton Rd,1
1673 Eddington Rd,1
30 Severance Circle,1


In [19]:
policeData_count.to_csv('../Data/processed/PD_count.csv')

In [20]:
len(policeData_count)

10549

In [21]:
policeData_count.reset_index(inplace=True)
policeData_count_shp = pd.merge(policeData_count, geocodes, how='inner', left_on='address1', right_on='address')
policeData_count_shp.drop('address1', axis=1, inplace=True)
policeData_count_shp['latitude'] = [float(str(row['coordinate']).split(',')[0][1:]) for ix,row in policeData_count_shp.iterrows()]
policeData_count_shp['longitude'] = [float(str(row['coordinate']).split(',')[1][1:-1]) for ix,row in policeData_count_shp.iterrows()]
policeData_count_shp.head()



,PD_Count,address,coordinate,latitude,longitude
0,2,3186 Euclid Heights Blvd,"(41.51042340000001, -81.5656856)",41.510423,-81.565686
1,2,4075 Monticello Blvd,"(41.536151, -81.5335763)",41.536151,-81.533576
2,1,1028 Greyton Rd,"(41.5321315, -81.5594579)",41.532131,-81.559458
3,1,1673 Eddington Rd,"(41.513896, -81.5765768)",41.513896,-81.576577
4,1,30 Severance Circle,"(41.5157586, -81.5551966)",41.515759,-81.555197


In [23]:
len(policeData_count_shp)

10549

In [24]:
geometry = gpd.GeoSeries([Point(xy) for xy in zip(policeData_count_shp.longitude, policeData_count_shp.latitude)])
policeData_count_shp = gpd.GeoDataFrame(policeData_count_shp, geometry=geometry)
policeData_count_shp.crs = {'init' :'epsg:4326'}
policeData_count_shp.dropna(subset=['geometry'], inplace=True)
policeData_count_shp.drop(['coordinate', 'latitude', 'longitude'], axis=1, inplace=True)
policeData_count_shp.head()

,PD_Count,address,geometry
0,2,3186 Euclid Heights Blvd,POINT (-81.56568559999999 41.51042340000001)
1,2,4075 Monticello Blvd,POINT (-81.53357629999999 41.536151)
2,1,1028 Greyton Rd,POINT (-81.5594579 41.5321315)
3,1,1673 Eddington Rd,POINT (-81.5765768 41.513896)
4,1,30 Severance Circle,POINT (-81.5551966 41.5157586)


In [25]:
policeData_count_shp.to_file('../Data/processed/shapefiles/PD_count.shp')

In [116]:
#pd.DataFrame(policeData.address1.unique()).to_csv('../Data/pd_geocodes.csv')